# Pràctica 4

In [1]:
from datasets import load_dataset

dataset = load_dataset("projecte-aina/catalan_general_crawling")

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/catalan_general_crawling contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/catalan_general_crawling
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [2]:
dataset = dataset['train']

# i want to see the first 10 examples
for i in range(10):
    print(dataset[i])
    print()

{'text': 'Reduïu els costos dels processos administratius al vostre organisme públic\nEviteu els desplaçaments i pèrdua de temps als ciutadans en les seves gestions\nOferiu una administració més transparent a ciutadans i empreses\nEns grans i petits experimenten aquesta transformació amb èxit, gràcies al suport de l\'AOC\nDepartament de Sistemes d\'Informació i Processos\n" Via Oberta ens ha permès fer efectiu el dret dels ciutadans a no aportar documents, eliminant paper i simplificant procediments"\n" e.FACT proporciona informació indispensable per a la realització de les auditories del registre comptable de factures de les Administracions Públiques Catalanes"\nCoordinador del departament d\'Informàtica\n"El servei VIA OBERTA és el que ha aportat majors avantatges per als ciutadans"\n"Amb l\' e-NOTUM hem escurçat els procediments en 12 dies, quasi un 40% menys!"\nCoordinadora d\'organització de persones i e-administració\n" Via Oberta ofereix millores per als ciutadans al no haver d\

Obtenim diferents mides de datasets

In [3]:
# i want to get the first 100 MB of the dataset
size = 0
for i in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 100000000:
        print(i)
        break

dataset_100MB = dataset.select(list(range(i)))


# i want to get the first 500 MB of the dataset
size = 0
for j in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 500000000:
        print(j)
        break

dataset_500MB = dataset.select(list(range(j)))


# i want to get the first 1 GB of the dataset
size = 0
for g in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 1000000000:
        print(g)
        break

dataset_1GB = dataset.select(list(range(g)))

57333
369003
738007


## Preprocessament

In [4]:
import gensim
from gensim.models import Word2Vec
import re

# Funció per netejar i tokenitzar el text
def preprocess_text(text):
    # Neteja el text: treu caràcters no desitjats, converteix a minúscules, etc.
    text = re.sub(r'\W+', ' ', text)  # Substitueix caràcters no alfanumèrics per espais
    text = text.lower()  # Converteix a minúscules
    words = text.split()  # Tokenitza
    return words

# Carrega el fitxer de text i processa'l
corpus_100MB = []
for line in dataset_100MB:
    words = preprocess_text(line['text'])
    if words:  # Assegura't que la línia no està buida
        corpus_100MB.append(words)

# Comprova algunes línies per assegurar-te que tot està bé
print(corpus_100MB[:2])


[['reduïu', 'els', 'costos', 'dels', 'processos', 'administratius', 'al', 'vostre', 'organisme', 'públic', 'eviteu', 'els', 'desplaçaments', 'i', 'pèrdua', 'de', 'temps', 'als', 'ciutadans', 'en', 'les', 'seves', 'gestions', 'oferiu', 'una', 'administració', 'més', 'transparent', 'a', 'ciutadans', 'i', 'empreses', 'ens', 'grans', 'i', 'petits', 'experimenten', 'aquesta', 'transformació', 'amb', 'èxit', 'gràcies', 'al', 'suport', 'de', 'l', 'aoc', 'departament', 'de', 'sistemes', 'd', 'informació', 'i', 'processos', 'via', 'oberta', 'ens', 'ha', 'permès', 'fer', 'efectiu', 'el', 'dret', 'dels', 'ciutadans', 'a', 'no', 'aportar', 'documents', 'eliminant', 'paper', 'i', 'simplificant', 'procediments', 'e', 'fact', 'proporciona', 'informació', 'indispensable', 'per', 'a', 'la', 'realització', 'de', 'les', 'auditories', 'del', 'registre', 'comptable', 'de', 'factures', 'de', 'les', 'administracions', 'públiques', 'catalanes', 'coordinador', 'del', 'departament', 'd', 'informàtica', 'el', 's

## Entrenament model de similitud de Text Semàntic

In [5]:
from datasets import load_dataset

text_semantic = load_dataset("projecte-aina/sts-ca")

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
from gensim.models import word2vec
model_100MB = word2vec.Word2Vec(corpus_100MB, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
model_100MB.save("model_100MB")
# Obtenir un word-vector

In [7]:
model_100MB = Word2Vec.load("model_100MB")

In [8]:
# Exemple: trobar paraules similars
similar_words = model_100MB.wv.most_similar('paraula')
print(similar_words)

[('frase', 0.7243554592132568), ('lletra', 0.6709128618240356), ('grafia', 0.6292251348495483), ('locució', 0.5762051343917847), ('afirmació', 0.5579834580421448), ('pronúncia', 0.556050717830658), ('recomanació', 0.5452941656112671), ('versió', 0.5388414263725281), ('piulada', 0.5328034162521362), ('conjunció', 0.5324693322181702)]


In [37]:
text_semantic_train = text_semantic['train']

In [38]:
text_semantic_test = text_semantic['test']

In [39]:
text_semantic_train[0]

{'sentence1': 'Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària',
 'sentence2': 'Creen la menció M. Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència universitària',
 'label': 3.5}

In [40]:
# Preprcessament del text_semantic
corpus_semantic = []
semantic_score = []
for line in text_semantic_train:
    frase1 = preprocess_text(line['sentence1'])
    frase2 = preprocess_text(line['sentence2'])

    corpus_semantic.append((frase1, frase2))
    semantic_score.append(line['label'])

In [44]:
corpus_semantic_test = []
semantic_score_test = []
for line in text_semantic_test:
    frase1 = preprocess_text(line['sentence1'])
    frase2 = preprocess_text(line['sentence2'])

    corpus_semantic_test.append((frase1, frase2))
    semantic_score_test.append(line['label'])

In [12]:
vocabulari = {}

for frases in corpus_semantic:
    for frase in frases:
        for paraula in frase:
            if paraula in vocabulari:
                vocabulari[paraula] += 1
            else:
                vocabulari[paraula] = 1


In [13]:
vocabulari_reduit = {palabra for palabra, frecuencia in vocabulari.items() if frecuencia >= 10}


### Model Baseline

In [14]:
import tensorflow as tf

def build_and_compile_model(input_length: int = 100, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16) -> tf.keras.Model:
    
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)
    
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True)
    
    lstm = tf.keras.layers.LSTM(hidden_size)
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)
    
    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 16)   │     16,000 │ input_layer[0][0… │
│ (Embedding)         │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 100)       │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 100)       │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     20,736 │ embedding[0][0],  │
│                     │                   │            │ not_equal[0][0],  │
│                     │                   │            │ embedding[1][0],  │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128)       │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ lstm[1][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      8,256 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,057 (176.00 KB)

 Trainable params: 45,057 (176.00 KB)

 Non-trainable params: 0 (0.00 B)

### 1. One-Hot

In [15]:
import tensorflow as tf

def build_and_compile_model(input_length: int = 764, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16) -> tf.keras.Model:
    
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)
    
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True)
    
    lstm = tf.keras.layers.LSTM(hidden_size)
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)
    
    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 764)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 764)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 764, 16)   │     16,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 764)       │          0 │ input_layer_2[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 764)       │          0 │ input_layer_3[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     20,736 │ embedding_1[0][0… │
│                     │                   │            │ not_equal_2[0][0… │
│                     │                   │            │ embedding_1[1][0… │
│                     │                   │            │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 128)       │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ lstm_1[1][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         65 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,057 (176.00 KB)

 Trainable params: 45,057 (176.00 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
import tensorflow as tf
def build_and_compile_model(hidden_size: int = 64) -> tf.keras.Model:
  model = tf.keras.Sequential([
      tf.keras.layers.Concatenate(axis=-1, ),
      tf.keras.layers.Dense(hidden_size, activation='relu'),
      tf.keras.layers.Dense(1)
  ])
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ concatenate_5 (Concatenate)     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [52]:
import numpy as np

# Función para calcular la representación one-hot de un texto
def compute_one_hot_encoding(text, vocabulari):
    """
    Genera la representación one-hot de un texto utilizando un vocabulario predefinido.
    """
    one_hot_vector = np.zeros(len(vocabulari))
    vocabulario_list = list(vocabulari)
    for palabra in text:
        if palabra in vocabulari:
            index = vocabulario_list.index(palabra)
            one_hot_vector[index] = 1
        
    return one_hot_vector


In [53]:
# Passar les frases de corpus_semantic a vectors
one_hot_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(one_hot_corpus_semantic):
    one_hot_corpus_semantic[i] = (compute_one_hot_encoding(frase1, vocabulari_reduit), 
                                    compute_one_hot_encoding(frase2, vocabulari_reduit))

In [54]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in one_hot_corpus_semantic])
vectors_2 = np.array([pair[1] for pair in one_hot_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
vectors_1 = np.clip(vectors_1, 0, 1)
vectors_2 = np.clip(vectors_2, 0, 1)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 1.7375 - val_loss: 0.7943
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.7853 - val_loss: 0.7524
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6658 - val_loss: 0.7523
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5912 - val_loss: 0.7481
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.5544 - val_loss: 0.7425
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.5000 - val_loss: 0.7424
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.4337 - val_loss: 0.7373
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.4066 - val_loss: 0.7502
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.3877 - val_loss: 0.7488
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3574 - val_loss: 0.7587
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3502 - val_loss: 0.7401
Epoch 12/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3376 - va

In [56]:
from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
one_hot_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(one_hot_corpus_semantic_test):
    one_hot_corpus_semantic_test[i] = (compute_one_hot_encoding(frase1, vocabulari_reduit), 
                                    compute_one_hot_encoding(frase2, vocabulari_reduit))

vectors_1 = np.array([pair[0] for pair in one_hot_corpus_semantic_test])
vectors_2 = np.array([pair[1] for pair in one_hot_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Error cuadrático medio (MSE): 1.1408123492599302
Coeficiente de determinación (R^2): -0.5109907388687134


### 2. Models Word2Vec pre-entrenats

In [60]:
import tensorflow as tf

def build_and_compile_model(input_length: int = 100, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16) -> tf.keras.Model:
    
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)
    
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True)
    
    lstm = tf.keras.layers.LSTM(hidden_size)
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)
    
    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 100, 16)   │     16,000 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_6         │ (None, 100)       │          0 │ input_layer_6[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_7         │ (None, 100)       │          0 │ input_layer_7[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 64)        │     20,736 │ embedding_3[0][0… │
│                     │                   │            │ not_equal_6[0][0… │
│                     │                   │            │ embedding_3[1][0… │
│                     │                   │            │ not_equal_7[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 128)       │          0 │ lstm_3[0][0],     │
│ (Concatenate)       │                   │            │ lstm_3[1][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │      8,256 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 1)         │         65 │ dense_12[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,057 (176.00 KB)

 Trainable params: 45,057 (176.00 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Word2Vec + Mean 
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector


# Word2Vec + Mean Ponderada
def compute_weighted_mean_embedding(text, model, word2tfidf, embedding_dim):
    """
    Utilitza una ponderació per a cada vector de paraula, com ara la freqüència 
    inversa del document (TF-IDF), per calcular una mitjana ponderada dels vectors.
    """
    vectors = [model[word] * word2tfidf[word] for word in text if word in model and word in word2tfidf]
    if vectors:
        weighted_mean_vector = np.mean(vectors, axis=0)
    else:
        weighted_mean_vector = np.zeros(embedding_dim)
    return weighted_mean_vector

In [25]:
# Passar les frases de corpus_semantic a vectors
mean_embbeding_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(mean_embbeding_corpus_semantic):
    mean_embbeding_corpus_semantic[i] = (compute_mean_embedding(frase1, model_100MB.wv, 100), 
                                        compute_mean_embedding(frase2, model_100MB.wv, 100))

In [61]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in mean_embbeding_corpus_semantic])
vectors_2 = np.array([pair[1] for pair in mean_embbeding_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
vectors_1 = np.clip(vectors_1, 0, 999)
vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

(2073, 100)
(2073, 100)
(2073,)
Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 30s 113ms/step - loss: 1.0416 - val_loss: 0.6647
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - loss: 0.6796 - val_loss: 0.6506
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 89ms/step - loss: 0.6749 - val_loss: 0.6265
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step - loss: 0.6844 - val_loss: 0.6309
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 8s 148ms/step - loss: 0.6500 - val_loss: 0.6248
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 6s 124ms/step - loss: 0.7021 - val_loss: 0.6774
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 11s 132ms/step - loss: 0.7045 - val_loss: 0.6253
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step - loss: 0.6799 - val_loss: 0.6241
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 130ms/step - loss: 0.6634 - val_loss: 0.6213
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 11s 137ms/step - loss: 0.6674 - val_loss: 0.6254
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 7s 132ms/step - loss: 0.7132 - val_loss: 0.6227
Epoch 12/50
52/52 ━

In [62]:
from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
mean_embbeding_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(mean_embbeding_corpus_semantic_test):
    mean_embbeding_corpus_semantic_test[i] = (
        compute_mean_embedding(frase1, model_100MB.wv, 100),
        compute_mean_embedding(frase2, model_100MB.wv, 100)
    )

vectors_1 = np.array([pair[0] for pair in mean_embbeding_corpus_semantic_test])
vectors_2 = np.array([pair[1] for pair in mean_embbeding_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step
Error cuadrático medio (MSE): 0.7948092671741267
Coeficiente de determinación (R^2): -0.05271422863006592


### 3. spaCy

In [ ]:
!py -m spacy download ca_core_news_md

     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     --------------------------------------- 0.0/49.2 MB 217.9 kB/s eta 0:03:46
     --------------------------------------- 0.0/49.2 MB 245.8 kB/s eta 0:03:21
     --------------------------------------- 0.1/49.2 MB 599.1 kB/s eta 0:01:22
     ---------------------------------------- 0.4/49.2 MB 1.8 MB/s eta 0:00:27
      --------------------------------------- 0.9/49.2 MB 3.3 MB/s eta 0:00:15
     - -------------------------------------- 2.0/49.2 MB 6.2 MB/s eta 0:00:08
     -- ------------------------------------- 2.9/49.2 MB 8.0 MB/s eta 0:00:06
     --- ------------------------------------ 3.8/49.2 MB 9.4 MB/s eta 0:00:05
     --- ------------------------------------ 4.8/49.2 MB 10.6 MB/s eta 0:00:05
     --- ------------------------------------ 4.9/49.2 MB 10.4 MB/s e

In [27]:
import spacy

nlp_md = spacy.load('ca_core_news_md')

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [63]:
def spacy_mean_embedding(text1, text2, model):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    text1 = ' '.join(text1)
    text2 = ' '.join(text2)
    frase1 = model(text1).vector
    frase2 = model(text2).vector
    
    return frase1, frase2

In [64]:
# Passar les frases de corpus_semantic a vectors
spacy_mean_embbeding_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(spacy_mean_embbeding_corpus_semantic):
    
    frase1, frase2 = spacy_mean_embedding(frase1, frase2, nlp_md)
    spacy_mean_embbeding_corpus_semantic[i] = (frase1, frase2)

In [65]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in spacy_mean_embbeding_corpus_semantic])
vectors_2 = np.array([pair[1] for pair in spacy_mean_embbeding_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
#vectors_1 = np.clip(vectors_1, 0, 999)
#vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

(2073, 300)
(2073, 300)
(2073,)
Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 25s 333ms/step - loss: 0.7655 - val_loss: 0.6411
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 19s 360ms/step - loss: 0.6877 - val_loss: 0.6320
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 22s 376ms/step - loss: 0.6797 - val_loss: 0.6466
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 19s 370ms/step - loss: 0.7062 - val_loss: 0.6940
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 375ms/step - loss: 0.6924 - val_loss: 0.6324
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 19s 372ms/step - loss: 0.6423 - val_loss: 0.6221
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 380ms/step - loss: 0.6710 - val_loss: 0.6314
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 15s 276ms/step - loss: 0.6761 - val_loss: 0.6448
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 13s 256ms/step - loss: 0.6416 - val_loss: 0.6529
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 13s 254ms/step - loss: 0.6720 - val_loss: 0.6706
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 13s 257ms/step - loss: 0.6504 - val_loss: 0.6472
Epoch 12/

In [66]:
from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
spacy_mean_embbeding_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(spacy_mean_embbeding_corpus_semantic_test):
    
    frase1, frase2 = spacy_mean_embedding(frase1, frase2, nlp_md)
    spacy_mean_embbeding_corpus_semantic_test[i] = (frase1, frase2)

vectors_1 = np.array([pair[0] for pair in spacy_mean_embbeding_corpus_semantic_test])
vectors_2 = np.array([pair[1] for pair in spacy_mean_embbeding_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step
Error cuadrático medio (MSE): 0.9937982237121572
Coeficiente de determinación (R^2): -0.3162723779678345


#### 4.RoBERTa

In [75]:
!py -m spacy download ca_core_news_trf

     ---------------------------------------- 0.0/457.1 MB ? eta -:--:--
     ---------------------------------------- 0.2/457.1 MB 6.9 MB/s eta 0:01:06
     ---------------------------------------- 0.3/457.1 MB 6.3 MB/s eta 0:01:13
     ---------------------------------------- 0.6/457.1 MB 4.6 MB/s eta 0:01:40
     ---------------------------------------- 0.6/457.1 MB 4.6 MB/s eta 0:01:40
     ---------------------------------------- 0.9/457.1 MB 3.6 MB/s eta 0:02:06
     ---------------------------------------- 1.3/457.1 MB 4.5 MB/s eta 0:01:42
     ---------------------------------------- 1.6/457.1 MB 4.8 MB/s eta 0:01:35
     ---------------------------------------- 1.8/457.1 MB 5.0 MB/s eta 0:01:31
     ---------------------------------------- 2.0/457.1 MB 4.6 MB/s eta 0:01:39
     ---------------------------------------- 2.4/457.1 MB 5.0 MB/s eta 0:01:31
     ---------------------------------------- 2.6/457.1 MB 4.9 MB/s eta 0:01:33
     ---------------------------------------- 2

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [76]:
import spacy

nlp_trf = spacy.load('ca_core_news_trf')

In [103]:
frase1 = 'atorga per primer cop les mencions encarna sanahuja a la inclusió de la perspectiva de gènere en docència universitària'
doc = nlp_trf(frase1)
embeddings = doc._.trf_data.last_hidden_layer_state
print(embeddings[-1])

Ragged(data=array([[ 2.09487468e-01, -6.82299376e-01,  6.87269866e-01,
        -2.15709984e-01, -1.83599502e-01,  1.96781412e-01,
        -3.10865909e-01,  5.01355380e-02,  2.09747404e-01,
         4.76011753e-01, -1.53212249e-01,  5.28716087e-01,
         3.63347411e-01, -3.85383755e-01,  1.04330353e-01,
         3.42985511e-01,  7.03844368e-01,  5.85401282e-02,
        -7.82101750e-02, -1.23748779e-02,  9.58655953e-01,
        -2.27109060e-01, -3.11816663e-01,  3.88369352e-01,
        -3.64216655e-01, -5.47646403e-01, -9.23638940e-01,
        -6.52197957e-01,  1.06132418e-01, -8.77194107e-02,
        -1.60520092e-01, -6.95155263e-01,  1.77061737e-01,
         1.20308086e-01, -2.87298769e-01, -2.34697625e-01,
        -7.32626542e-02, -1.40464321e-01, -2.27952570e-01,
        -5.01841426e-01,  2.91805565e-02,  2.90764600e-01,
        -2.10933387e-01,  7.32212216e-02, -8.17313492e-01,
         6.53707981e-03, -1.85016036e-01,  1.21130973e-01,
        -4.40823376e-01,  4.19431001e-01, -7

In [93]:
frase1

'atorga per primer cop les mencions encarna sanahuja a la inclusió de la perspectiva de gènere en docència universitària'

In [90]:
print(dir(doc._.trf_data))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'all_hidden_layer_states', 'all_outputs', 'embedding_layer', 'from_dict', 'last_hidden_layer_state', 'last_layer_only', 'num_outputs', 'to_dict']


In [79]:

# Tu frase
frase = "Tu frase aquí"

def spacy_trf_CLS(frase1, frase2, model):# Procesar la frase
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    doc1 = model(frase1)
    doc2 = model(frase2)

    # Obtener el embedding CLS
    embedding_CLS1 = doc1._.trf_data.tensors[-1][0]
    embedding_CLS2 = doc2._.trf_data.tensors[-1][0]
    
    return embedding_CLS1, embedding_CLS2


In [81]:
spacy_trf_CLS_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(spacy_trf_CLS_corpus_semantic):
    
    frase1, frase2 = spacy_trf_CLS(frase1, frase2, nlp_trf)
    spacy_trf_CLS_corpus_semantic[i] = (frase1, frase2)

AttributeError: 'DocTransformerOutput' object has no attribute 'tensors'

### 5. RoBERTa fine-tuned

In [67]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [70]:
def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

In [74]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
model_name = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = AutoModel.from_pretrained(model_name)

# Convert sentence pairs to embeddings using the transformer model
def get_embeddings(sentence_pairs):
    prepared_pairs = prepare(sentence_pairs)
    embeddings = []
    for pair in prepared_pairs:
        inputs = tokenizer(pair, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = transformer_model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return embeddings

fine_tuned_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(fine_tuned_corpus_semantic):
    fine_tuned_corpus_semantic[i] = (' '.join(frase1), ' '.join(frase2))

embeddings = get_embeddings(fine_tuned_corpus_semantic)

Some weights of RobertaModel were not initialized from the model checkpoint at projecte-aina/roberta-base-ca-v2-cased-sts and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [73]:


# Separate embeddings into two arrays for the sentence pairs
vectors_1 = np.array([embedding[:len(embedding)//2] for embedding in embeddings])
vectors_2 = np.array([embedding[len(embedding)//2:] for embedding in embeddings])

# Assuming 'semantic_score' is already defined as the target similarity scores
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

# Define and compile the Keras model
def build_and_compile_model(input_length: int, hidden_size: int = 64) -> tf.keras.Model:
    input_1, input_2 = tf.keras.Input(shape=(input_length,)), tf.keras.Input(shape=(input_length,))
    
    dense = tf.keras.layers.Dense(hidden_size, activation='relu')
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    _dense_1, _dense_2 = dense(input_1), dense(input_2)
    _concatenated = concatenate([_dense_1, _dense_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.01))
    return model

input_length = vectors_1.shape[1]
baseline_model = build_and_compile_model(input_length)
baseline_model.summary()

# Train the model
baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at projecte-aina/roberta-base-ca-v2-cased-sts and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [ ]:
fine_runed_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(fine_runed_corpus_semantic):
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    fine_runed_corpus_semantic[i] = (frase1, frase2)


In [ ]:
predictions = pipe(prepare(fine_runed_corpus_semantic), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)

[{'label': 'SIMILARITY', 'score': 3.639443974101274}, {'label': 'SIMILARITY', 'score': 1.3343049254405437}, {'label': 'SIMILARITY', 'score': 3.6255003414699982}, {'label': 'SIMILARITY', 'score': 2.8571212701413073}, {'label': 'SIMILARITY', 'score': 1.7412681993979517}, {'label': 'SIMILARITY', 'score': 2.902458331608203}, {'label': 'SIMILARITY', 'score': 3.3925175786892536}, {'label': 'SIMILARITY', 'score': 2.7214751443641485}, {'label': 'SIMILARITY', 'score': 2.848001481281147}, {'label': 'SIMILARITY', 'score': 2.8075600107730767}, {'label': 'SIMILARITY', 'score': 2.9737861311233997}, {'label': 'SIMILARITY', 'score': 0.946479625573919}, {'label': 'SIMILARITY', 'score': 2.0864164114103474}, {'label': 'SIMILARITY', 'score': 3.390759591752991}, {'label': 'SIMILARITY', 'score': 3.119538929859854}, {'label': 'SIMILARITY', 'score': 2.835003453924068}, {'label': 'SIMILARITY', 'score': 1.6847840009938235}, {'label': 'SIMILARITY', 'score': 3.1470772247192316}, {'label': 'SIMILARITY', 'score': 2

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
mean_embbeding_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(mean_embbeding_corpus_semantic_test):
    mean_embbeding_corpus_semantic_test[i] = (
        compute_mean_embedding(frase1, model_100MB.wv, 100),
        compute_mean_embedding(frase2, model_100MB.wv, 100)
    )

vectors_1 = np.array([pair[0] for pair in mean_embbeding_corpus_semantic_test])
vectors_2 = np.array([pair[1] for pair in mean_embbeding_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)


### model amb embeddings entrenables

#### Random Embeddings

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

vectors_1 =[pair[0] for pair in mean_embbeding_corpus_semantic]
vectors_2 =[pair[1] for pair in mean_embbeding_corpus_semantic]

# Tokenització i creació del vocabulari
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_1 + texts_2)
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
word_index = tokenizer.word_index

# Padding de les seqüències
max_length = 10
data_1 = pad_sequences(sequences_1, maxlen=max_length)
data_2 = pad_sequences(sequences_2, maxlen=max_length)

# Paràmetres del model
vocab_size = len(word_index) + 1
embedding_dim = 100
hidden_size = 64
input_length = max_length

# Creació d'embeddings aleatoris
random_embeddings = np.random.uniform(-1, 1, (vocab_size, embedding_dim))

# Creació d'embeddings de Word2Vec
texts_combined = [text.split() for text in texts_1 + texts_2]
word2vec_model = Word2Vec(texts_combined, vector_size=embedding_dim, window=5, min_count=1, workers=4)
word2vec_embeddings = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        word2vec_embeddings[i] = word2vec_model.wv[word]

# Entrenament del model amb embeddings aleatoris
model_random = build_and_compile_model(random_embeddings, input_length=max_length, hidden_size=hidden_size)
model_random.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Entrenament del model amb embeddings de Word2Vec
model_word2vec = build_and_compile_model(word2vec_embeddings, input_length=max_length, hidden_size=hidden_size)
model_word2vec.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Avalua els resultats dels models
loss_random = model_random.evaluate([data_1, data_2], labels)
loss_word2vec = model_word2vec.evaluate([data_1, data_2], labels)

print(f'Random Embeddings - Loss: {loss_random}')
print(f'Word2Vec Embeddings - Loss: {loss_word2vec}')


In [ ]:
import tensorflow as tf
def build_and_compile_model(
        input_length: int = 10, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16,
) -> tf.keras.Model:
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32, ), tf.keras.Input((input_length, ), dtype=tf.int32, )
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True, )
    pooling = tf.keras.layers.GlobalAveragePooling1D()
    concatenate = tf.keras.layers.Concatenate(axis=-1, )
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    # Pass through the layers
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    _embedded_1, _embedded_2 = embedding(input_1, ), embedding(input_2, )
    _pooled_1, _pooled_2 = pooling(_embedded_1, mask=_input_mask_1), pooling(_embedded_2, mask=_input_mask_2)
    _concatenated = concatenate((_pooled_1, _pooled_2, ))
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2, ), outputs=_output, )
    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
    return model

#### Word2Vec

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

# Exemple de dades
texts_1 = ["aquest és un text d'exemple", "un altre text"]
texts_2 = ["aquest és un altre text", "un text diferent"]
labels = [0.8, 0.4]  # Similitud entre les frases

# Tokenització i creació del vocabulari
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_1 + texts_2)
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
word_index = tokenizer.word_index

# Padding de les seqüències
max_length = 10
data_1 = pad_sequences(sequences_1, maxlen=max_length)
data_2 = pad_sequences(sequences_2, maxlen=max_length)

# Paràmetres del model
vocab_size = len(word_index) + 1
embedding_dim = 100
hidden_size = 64
input_length = max_length

# Creació d'embeddings de Word2Vec
texts_combined = [text.split() for text in texts_1 + texts_2]
word2vec_model = Word2Vec(texts_combined, vector_size=embedding_dim, window=5, min_count=1, workers=4)
word2vec_embeddings = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        word2vec_embeddings[i] = word2vec_model.wv[word]

def build_and_compile_model(embedding_matrix, input_length: int = 100, hidden_size: int = 64) -> tf.keras.Model:
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)

    # Define Layers
    embedding = tf.keras.layers.Embedding(
        vocab_size, embedding_dim, weights=[embedding_matrix], input_length=input_length, mask_zero=True, trainable=True)

    lstm = tf.keras.layers.LSTM(hidden_size)

    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)

    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)

    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)

    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Entrenament del model amb embeddings de Word2Vec
model_word2vec = build_and_compile_model(word2vec_embeddings, input_length=max_length, hidden_size=hidden_size)
model_word2vec.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Avalua els resultats dels models
loss_word2vec = model_word2vec.evaluate([data_1, data_2], labels)

print(f'Word2Vec Embeddings - Loss: {loss_word2vec}')


### Prova

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Defineix funcions per calcular els embeddings de frases
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector

def compute_weighted_mean_embedding(text, model, word2tfidf, embedding_dim):
    """
    Utilitza una ponderació per a cada vector de paraula, com ara la freqüència 
    inversa del document (TF-IDF), per calcular una mitjana ponderada dels vectors.
    """
    vectors = [model[word] * word2tfidf[word] for word in text if word in model and word in word2tfidf]
    if vectors:
        weighted_mean_vector = np.mean(vectors, axis=0)
    else:
        weighted_mean_vector = np.zeros(embedding_dim)
    return weighted_mean_vector

In [ ]:
# Exemple de corpus
corpus = ["Aquest és un text de exemple", "Aquí hi ha un altre text"]

# Prepara els vectors TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [ ]:
# Exemple d'ús amb embeddings pre-entrenats (Word2Vec)
from gensim.models import KeyedVectors
import tensorflow as tf

# Carrega el model pre-entrenat (substitueix 'path_to_model' amb la ruta real)
# word2vec_model = KeyedVectors.load_word2vec_format('path_to_model', binary=True)

# Defineix la longitud de les seqüències i la dimensió dels embeddings
input_length = 10
embedding_dim = model_100MB.vector_size

# Prepara les dades d'entrenament
X_train1 = np.array([compute_mean_embedding(text.split(), model_100MB.wv, embedding_dim) for text in corpus])
X_train2 = np.array([compute_weighted_mean_embedding(text.split(), model_100MB.wv, word2tfidf, embedding_dim) for text in corpus])
y_train = np.random.rand(len(corpus), 1)

# Defineix el model de Keras (ajusta les dimensions segons la longitud de les seqüències i la dimensió dels embeddings)
def build_and_compile_model(input_dim, hidden_size=64):
    input_1 = tf.keras.Input(shape=(input_dim,))
    input_2 = tf.keras.Input(shape=(input_dim,))
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)([input_1, input_2])
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')(concatenate)
    output = tf.keras.layers.Dense(1)(hidden)
    
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

# Construir i compilar el model
m = build_and_compile_model(input_dim=embedding_dim)

# Entrenar el model
m.fit([X_train1, X_train2], y_train, epochs=10, batch_size=32)

# Exemple de predicció
input_data1 = compute_mean_embedding("Aquest és un text de exemple".split(), model_100MB.wv, embedding_dim).reshape(1, -1)
input_data2 = compute_weighted_mean_embedding("Aquí hi ha un altre text".split(), model_100MB.wv, word2tfidf, embedding_dim).reshape(1, -1)

# Predir la similitud
y_pred = m.predict([input_data1, input_data2])
print(y_pred)

Epoch 1/10
1/1 [==============================] - 0s 334ms/step - loss: 0.7557
Epoch 2/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1112
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4268
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.5486
Epoch 5/10
1/1 [==============================] - 0s 3ms/step - loss: 0.4800
Epoch 6/10
1/1 [==============================] - 0s 5ms/step - loss: 0.3008
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 0.0543
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2087
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 0.2880
Epoch 10/10
1/1 [==============================] - 0s 55ms/step
[[0.10263795]]


In [ ]:
y_train

array([[0.24829134],
       [0.49559584]])